In [1]:
pip install delta-spark==3.3.2 dotenv

  Using cached pyspark-3.5.7-py2.py3-none-any.whl
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.0
    Can't uninstall 'pyspark'. No files were found to uninstall.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable
from datetime import datetime

In [3]:
from dotenv import load_dotenv
import os
load_dotenv('/opt/workspace/.env')
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT_DOCKER")
MINIO_ACCESS = os.getenv("MINIO_ROOT_USER")
MINIO_SECRET = os.getenv("MINIO_ROOT_PASSWORD")
                           
today = datetime.now().strftime("%Y/%m/%d")
BRONZE_PATH = f"s3a://bronze/posicao/{today}/"
SILVER_PATH = "s3a://silver/posicao/"
print(f"Buscando arquivos arquivos em: {BRONZE_PATH}")

Buscando arquivos arquivos em: s3a://bronze/posicao/2025/11/08/


In [4]:
builder = (
    SparkSession.builder.appName("BronzeToSilver_Delta")
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT)
    .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS)
    .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET)
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    # Delta Lake
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [5]:
schema = StructType([
    StructField("hr", StringType(), True),
    StructField("l", ArrayType(
        StructType([
            StructField("c", StringType(), True),
            StructField("cl", IntegerType(), True),
            StructField("sl", IntegerType(), True),
            StructField("lt0", StringType(), True),
            StructField("lt1", StringType(), True),
            StructField("qv", IntegerType(), True),
            StructField("vs", ArrayType(
                StructType([
                    StructField("p", IntegerType(), True),
                    StructField("a", BooleanType(), True),
                    StructField("ta", StringType(), True),
                    StructField("py", DoubleType(), True),
                    StructField("px", DoubleType(), True),
                ])
            ), True)
        ])
    ), True)
])


In [6]:
# Pega o arquivo mais recente
files_df = spark.read.format("binaryFile").load(BRONZE_PATH)
latest_file_row = (
    files_df.orderBy(F.col("modificationTime").desc())
    .select("path")
    .limit(1)
    .collect()
)
latest_file = latest_file_row[0].path
print(latest_file)

s3a://bronze/posicao/2025/11/08/posicao_20251108_014717.json


In [7]:
df_raw = spark.read.option("mode", "PERMISSIVE").schema(schema).json(latest_file)

if df_raw.isEmpty() or df_raw.filter(F.col("hr").isNotNull() & F.col("l").isNotNull()).isEmpty():
    print("⚠️ Arquivo vazio.")
    spark.stop()
    exit(0)

In [8]:
# 1. EXPLODE o array "l" linha
df_linhas = df_raw.selectExpr("hr", "inline(l)")
df_linhas.show(2, truncate=False)
df_linhas.printSchema()

+-----+-------+----+---+---------------+-----------+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|hr   |c      |cl  |sl |lt0            |lt1        |qv |vs                                                                                                                                                                                                                                                                                                                               |
+-----+-------+----+---+---------------+-----------+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
# 2. EXPLODE o array "vs" veículos
df_veiculos = df_linhas.selectExpr(
    "hr",
    "c as letreiro",
    "cl as codigo_linha",
    "sl as sentido",
    "lt0 as terminal_inicial",
    "lt1 as terminal_final",
    "qv",
    "inline(vs)"  # expande os veículos
)
df_veiculos.show(3, truncate=False)
df_veiculos.printSchema()

+-----+--------+------------+-------+----------------+--------------+---+-----+----+--------------------+-------------------+------------+
|hr   |letreiro|codigo_linha|sentido|terminal_inicial|terminal_final|qv |p    |a   |ta                  |py                 |px          |
+-----+--------+------------+-------+----------------+--------------+---+-----+----+--------------------+-------------------+------------+
|22:47|5164-21 |1616        |1      |PQ. IBIRAPUERA  |CID. LEONOR   |5  |64469|true|2025-11-08T01:47:06Z|-23.6524195        |-46.6465905 |
|22:47|5164-21 |1616        |1      |PQ. IBIRAPUERA  |CID. LEONOR   |5  |64734|true|2025-11-08T01:47:12Z|-23.647640000000003|-46.651212  |
|22:47|5164-21 |1616        |1      |PQ. IBIRAPUERA  |CID. LEONOR   |5  |64474|true|2025-11-08T01:46:51Z|-23.6463425        |-46.64646425|
+-----+--------+------------+-------+----------------+--------------+---+-----+----+--------------------+-------------------+------------+
only showing top 3 rows

ro

In [10]:
# 3. FILTRAR ONIBUS NAO REGULAR
df_filtrado = df_veiculos.filter(
    "codigo_linha IS NOT NULL AND NOT (codigo_linha < 1000 OR letreiro RLIKE 'GUIN|TEST|TST')"
)
print(f"Total de ônibus teste/guincho filtrados: {df_veiculos.count() - df_filtrado.count()}")

Total de ônibus teste/guincho filtrados: 1718


In [11]:
# 4. Selecionar e renomear colunas úteis
df_limpo = df_filtrado.select(
    "letreiro",
    "codigo_linha",
    "sentido",
    "terminal_inicial",
    "terminal_final",
    F.col("p").alias("codigo_veiculo"),
    F.col("a").alias("acessibilidade"),
    F.to_timestamp("ta").alias("ultima_atualizacao"),
    F.col("py").alias("latitude"),
    F.col("px").alias("longitude"),
    F.to_timestamp("hr").alias("hora_referencia"),
)
df_limpo.show(5, truncate=False)

+--------+------------+-------+----------------+--------------+--------------+--------------+-------------------+-------------------+------------+-------------------+
|letreiro|codigo_linha|sentido|terminal_inicial|terminal_final|codigo_veiculo|acessibilidade|ultima_atualizacao |latitude           |longitude   |hora_referencia    |
+--------+------------+-------+----------------+--------------+--------------+--------------+-------------------+-------------------+------------+-------------------+
|5164-21 |1616        |1      |PQ. IBIRAPUERA  |CID. LEONOR   |64469         |true          |2025-11-08 01:47:06|-23.6524195        |-46.6465905 |2025-11-08 22:47:00|
|5164-21 |1616        |1      |PQ. IBIRAPUERA  |CID. LEONOR   |64734         |true          |2025-11-08 01:47:12|-23.647640000000003|-46.651212  |2025-11-08 22:47:00|
|5164-21 |1616        |1      |PQ. IBIRAPUERA  |CID. LEONOR   |64474         |true          |2025-11-08 01:46:51|-23.6463425        |-46.64646425|2025-11-08 22:47:00

In [12]:
# 5. Remover duplicatas de registros (se houver)
df_dedup = df_limpo.dropDuplicates(["codigo_veiculo", "hora_referencia"])
print(f"Quantidade de registros || Antes: {df_limpo.count()} | Depois: {df_dedup.count()} | # de duplicatas: {df_limpo.count() - df_dedup.count()}")

Quantidade de registros || Antes: 5384 | Depois: 5384 | # de duplicatas: 0


In [13]:
# 6. Corrigir imprecisão floating point lat/long e adicionar metadados
df_final = (
    df_dedup
    .withColumn("latitude", F.round("latitude", 6))
    .withColumn("longitude", F.round("longitude", 6))
    .withColumn("data_ref", F.to_date("ultima_atualizacao"))
    .withColumn("ingest_timestamp", F.current_timestamp())
)
df_final.show(3, truncate=False)
df_final.printSchema()

+--------+------------+-------+------------------------+-----------------+--------------+--------------+-------------------+----------+----------+-------------------+----------+--------------------------+
|letreiro|codigo_linha|sentido|terminal_inicial        |terminal_final   |codigo_veiculo|acessibilidade|ultima_atualizacao |latitude  |longitude |hora_referencia    |data_ref  |ingest_timestamp          |
+--------+------------+-------+------------------------+-----------------+--------------+--------------+-------------------+----------+----------+-------------------+----------+--------------------------+
|414P-10 |1105        |1      |TERM. NORTE METRÔ CARRÃO|VL. INDUSTRIAL   |3117          |false         |2025-11-08 01:46:53|-23.610801|-46.529946|2025-11-08 22:47:00|2025-11-08|2025-11-08 01:50:15.346703|
|6027-10 |32858       |2      |TERM. STO. AMARO        |JD. GRAUNA       |7811          |false         |2025-11-08 01:46:43|-23.67923 |-46.706624|2025-11-08 22:47:00|2025-11-08|202

In [14]:
df_final.show(4, truncate=False)

+--------+------------+-------+------------------------+-----------------+--------------+--------------+-------------------+----------+----------+-------------------+----------+--------------------------+
|letreiro|codigo_linha|sentido|terminal_inicial        |terminal_final   |codigo_veiculo|acessibilidade|ultima_atualizacao |latitude  |longitude |hora_referencia    |data_ref  |ingest_timestamp          |
+--------+------------+-------+------------------------+-----------------+--------------+--------------+-------------------+----------+----------+-------------------+----------+--------------------------+
|414P-10 |1105        |1      |TERM. NORTE METRÔ CARRÃO|VL. INDUSTRIAL   |3117          |false         |2025-11-08 01:46:53|-23.610801|-46.529946|2025-11-08 22:47:00|2025-11-08|2025-11-08 01:50:15.792117|
|6027-10 |32858       |2      |TERM. STO. AMARO        |JD. GRAUNA       |7811          |false         |2025-11-08 01:46:43|-23.67923 |-46.706624|2025-11-08 22:47:00|2025-11-08|202

In [15]:
if DeltaTable.isDeltaTable(spark, SILVER_PATH):
    silver_table = DeltaTable.forPath(spark, SILVER_PATH)
    (
        silver_table.alias("tgt")
        .merge(
            df_final.alias("src"),
            "tgt.codigo_veiculo = src.codigo_veiculo AND tgt.hora_referencia = src.hora_referencia"
        )
        .whenNotMatchedInsertAll()
        .execute()
    )
    print("🔁 MERGE incremental concluído.")
else:
    (
        df_final.write.format("delta")
        .mode("overwrite")
        .partitionBy("data_ref")
        .save(SILVER_PATH)
    )
    print("🆕 Nova tabela Delta criada.")

🔁 MERGE incremental concluído.
